In [ ]:
!pip install datasets

In [ ]:
pip install -U bitsandbytes

In [ ]:
pip install einops

In [ ]:
pip install timm

# Imports & Constants

In [25]:
device = torch.device("cpu")

In [30]:
import numpy as np
import torch
import torchvision.transforms as T
from PIL import Image
from torchvision.transforms.functional import InterpolationMode
from transformers import AutoModel, AutoTokenizer
from datasets import load_dataset

# Constants for image normalization (ImageNet stats)
IMAGENET_MEAN = (0.485, 0.456, 0.406)
IMAGENET_STD = (0.229, 0.224, 0.225)

def build_transform(input_size):
    transform = T.Compose([
        T.Lambda(lambda img: img.convert('RGB') if img.mode != 'RGB' else img),
        T.Resize((input_size, input_size), interpolation=InterpolationMode.BICUBIC),
        T.ToTensor(),
        T.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
    ])
    return transform

def find_closest_aspect_ratio(aspect_ratio, target_ratios, width, height, image_size):
    best_ratio_diff = float('inf')
    best_ratio = (1, 1)
    area = width * height
    for ratio in target_ratios:
        target_aspect_ratio = ratio[0] / ratio[1]
        ratio_diff = abs(aspect_ratio - target_aspect_ratio)
        if ratio_diff < best_ratio_diff:
            best_ratio_diff = ratio_diff
            best_ratio = ratio
        elif ratio_diff == best_ratio_diff:
            if area > 0.5 * image_size * image_size * ratio[0] * ratio[1]:
                best_ratio = ratio
    return best_ratio

def dynamic_preprocess(image, min_num=1, max_num=12, image_size=448, use_thumbnail=False):
    orig_width, orig_height = image.size
    aspect_ratio = orig_width / orig_height

    target_ratios = set(
        (i, j) for n in range(min_num, max_num + 1)
        for i in range(1, n + 1)
        for j in range(1, n + 1)
        if i * j <= max_num and i * j >= min_num
    )
    target_ratios = sorted(target_ratios, key=lambda x: x[0] * x[1])
    target_aspect_ratio = find_closest_aspect_ratio(aspect_ratio, target_ratios, orig_width, orig_height, image_size)
    target_width = image_size * target_aspect_ratio[0]
    target_height = image_size * target_aspect_ratio[1]
    blocks = target_aspect_ratio[0] * target_aspect_ratio[1]

    resized_img = image.resize((target_width, target_height))
    processed_images = []
    for i in range(blocks):
        box = (
            (i % (target_width // image_size)) * image_size,
            (i // (target_width // image_size)) * image_size,
            ((i % (target_width // image_size)) + 1) * image_size,
            ((i // (target_width // image_size)) + 1) * image_size
        )
        split_img = resized_img.crop(box)
        processed_images.append(split_img)
    assert len(processed_images) == blocks
    if use_thumbnail and len(processed_images) != 1:
        thumbnail_img = image.resize((image_size, image_size))
        processed_images.append(thumbnail_img)
    return processed_images

def process_dataset_image(pil_image, input_size=448, max_num=12):
    """
    Given a PIL image (from the dataset), preprocess it into pixel_values
    suitable for the InternVL model.
    """
    # Ensure that the image passed is a PIL Image (if not, raise an error)
    if isinstance(pil_image, list):
        pil_image = pil_image[0]  # If the dataset provides the image as a list, take the first item

    transform = build_transform(input_size=input_size)
    # Dynamically split the image into tiles (if necessary)
    images = dynamic_preprocess(pil_image, image_size=input_size, use_thumbnail=True, max_num=max_num)
    pixel_values = [transform(img) for img in images]
    pixel_values = torch.stack(pixel_values).to(torch.bfloat16).to(device)
    return pixel_values



# Load Model and Tokenizer

In [31]:
# Load InternVL Chat model and tokenizer
path = 'OpenGVLab/InternVL-Chat-V1-5'
model = AutoModel.from_pretrained(
    path,
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    trust_remote_code=True
).eval().to(device)

tokenizer = AutoTokenizer.from_pretrained(path, trust_remote_code=True, use_fast=False)

Loading checkpoint shards:   0%|          | 0/11 [00:00<?, ?it/s]

# Helper Function to Remove Problem Statement

In [32]:
def remove_problem_from_prediction(prompt, prediction):
    """
    If the prediction begins with the prompt, remove it.
    """
    if prediction.startswith(prompt):
        return prediction[len(prompt):].strip()
    else:
        return prediction.strip()

# Load the “hiyouga/geometry3k” Dataset

In [13]:
# Load the dataset
ds = load_dataset("hiyouga/geometry3k")
dataset = ds["test"]

README.md:   0%|          | 0.00/2.38k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/41.6M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/5.83M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/11.8M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2101 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/300 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/601 [00:00<?, ? examples/s]

In [33]:
def evaluate_internvl_model(model, tokenizer, dataset, generation_config, device=device,
                            max_samples=None, input_size=448, max_num=12):
    """
    Evaluate the InternVL Chat model on a dataset where each sample contains:
      - "images": a PIL image,
      - "problem": a text problem statement,
      - "answer" or "ground_truth": the expected output.

    The function builds a prompt for each sample by prepending "<image>\n" to the problem,
    then calls the model's chat method and removes the prompt from the generated answer.

    Returns a list of dictionaries with the keys:
      "problem", "prediction", and "ground_truth".
    """
    results = []

    for idx, sample in enumerate(dataset):
        if max_samples is not None and idx >= max_samples:
            break

        # Extract the problem and ground truth
        problem = sample["problem"]
        ground_truth = sample.get("ground_truth", sample.get("answer", ""))

        # Process the image (assumed to be a PIL image)
        image = sample["images"]
        pixel_values = process_dataset_image(image, input_size=input_size, max_num=max_num)

        # Build a prompt for InternVL Chat
        # Here, we simply prepend "<image>\n" to indicate an image input
        prompt = '<image>\n' + problem

        # Generate response using the model's chat method
        # (For single-image inference, we pass in pixel_values)
        response, history = model.chat(tokenizer, pixel_values, prompt, generation_config,
                                       history=None, return_history=True)

        # Remove the prompt from the response if needed
        clean_response = remove_problem_from_prediction(prompt, response)

        results.append({
            "problem": problem,
            "prediction": clean_response,
            "ground_truth": ground_truth
        })

        if idx % 10 == 0:
            print(f"Processed sample {idx}")

    return results


In [39]:
generation_config = dict(max_new_tokens=10, do_sample=True)
results = evaluate_internvl_model(model, tokenizer, dataset, generation_config,
                                  max_samples=1, input_size=448, max_num=12)


# Display the results
for idx, item in enumerate(results):
    print(f"Sample {idx}:")
    print("Problem:", item["problem"])
    print("Prediction:", item["prediction"])
    print("Ground Truth:", item["ground_truth"])
    print("-" * 50)


Processed sample 0
Sample 0:
Problem: <image>In the figure, $\overline{A D}$ is perpendicular to $\overline{B C}$ and $\overline{A B}$ is perpendicular to $\overline{A C}$. What is $B C ?$
Prediction: The triangle that is right-angled at $A
Ground Truth: C
--------------------------------------------------
